In [1]:
%matplotlib inline
import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.contrib import slim
from tensorflow.contrib.learn import ModeKeys
from tensorflow.contrib.learn import learn_runner

In [2]:
TRAIN_SIZE = 18500
EVAL_SIZE = 2500
MEAN = 93.6877751708 
STD = 76.9036483865

In [3]:
def architecture(inputs, is_training, scope='MnistConvNet'):
    """Return the output operation following the network architecture.
    Args:
        inputs (Tensor): Input Tensor
        is_training (bool): True iff in training mode
        scope (str): Name of the scope of the architecture
    Returns:
         Logits output Op for the network.
    """
    print (inputs)
    with tf.variable_scope(scope):
        # Convolution Layer with 32 filters and a kernel size of 5
        conv1 = tf.layers.conv2d(inputs, 32, 5, activation=tf.nn.relu)
        conv1 = tf.layers.max_pooling2d(conv1, 4, 2)

        conv2 = tf.layers.conv2d(conv1, 64, 3, activation=tf.nn.relu)
        conv2 = tf.layers.max_pooling2d(conv2, 4, 2)

        conv3 = tf.layers.conv2d(conv1, 64, 3, activation=tf.nn.relu)
        conv3 = tf.layers.max_pooling2d(conv3, 4, 2)
        
        fc1 = tf.layers.flatten(conv3)

        fc1 = tf.layers.dense(fc1, 1024)
        fc1 = tf.layers.dropout(fc1, rate=0.5, training=is_training)

        # Output layer, class prediction
        out = tf.layers.dense(fc1, 12)

        return out
    
def nvidia_architecture(inputs, is_training, scope='Nvidia'):
    with tf.variable_scope(scope):
        net = tf.layers.conv2d(inputs, 24, 5, strides=(2, 2), activation=tf.nn.relu, name='conv1')
        net = tf.layers.conv2d(net, 36, 5, strides=(2, 2), activation=tf.nn.relu, name='conv2')
        net = tf.layers.conv2d(net, 48, 5, strides=(2, 2), activation=tf.nn.relu, name='conv3')
        net = tf.layers.conv2d(net, 64, 3, activation=tf.nn.relu, name='conv4')
        net = tf.layers.conv2d(net, 64, 3, activation=tf.nn.relu, name='conv5')
        
        net = tf.layers.flatten(net)
        net = tf.layers.dense(net, 1164, activation=tf.nn.relu, name='fc1')
        net = tf.layers.dropout(net, rate=0.7, training=is_training)
        
        net = tf.layers.dense(net, 100, activation=tf.nn.relu, name='fc2')
        net = tf.layers.dropout(net, rate=0.7, training=is_training)
        
        net = tf.layers.dense(net, 50, activation=tf.nn.relu, name='fc3')
        net = tf.layers.dropout(net, rate=0.7, training=is_training)
        
        net = tf.layers.dense(net, 20, activation=tf.nn.relu, name='fc4')
        
        # scaling based on sfmlearner
        predictions = 0.01 * tf.layers.dense(net, 12, activation=None, name='predictions')
        return predictions
    
def sfmlearner_architecture(inputs, is_training, scope='sfmlearner'):
    with tf.variable_scope(scope):
        regularizer = tf.contrib.layers.l2_regularizer(scale=0.05)
        net = tf.layers.conv2d(inputs, 16, 7, strides=(2, 2), activation=tf.nn.relu, name='conv1', kernel_regularizer=regularizer)
        net = tf.layers.conv2d(net, 32, 5, strides=(2, 2), activation=tf.nn.relu, name='conv2', kernel_regularizer=regularizer)
        net = tf.layers.conv2d(net, 64, 3, strides=(2, 2), activation=tf.nn.relu, name='conv3', kernel_regularizer=regularizer)
        net = tf.layers.conv2d(net, 128, 3, strides=(2, 2), activation=tf.nn.relu, name='conv4', kernel_regularizer=regularizer)
        net = tf.layers.conv2d(net, 256, 3, strides=(2, 2), activation=tf.nn.relu, name='conv5', kernel_regularizer=regularizer)
        net = tf.layers.conv2d(net, 256, 3, strides=(2, 2), activation=tf.nn.relu, name='conv6', kernel_regularizer=regularizer)
        net = tf.layers.conv2d(net, 256, 3, strides=(2, 2), activation=tf.nn.relu, name='conv7', kernel_regularizer=regularizer)
        predictions = tf.layers.dense(net, 12, activation=None, name='predictions')
        predictions = tf.reduce_mean(predictions, [1, 2])
        predictions = 0.01 * tf.reshape(predictions, [-1, ])
        
        return predictions
    
def sfmlearner_architecture2(inputs, is_training, scope='sfmlearner'):
    slim = tf.contrib.slim
    with tf.variable_scope('pose_exp_net') as sc:
        with slim.arg_scope([slim.conv2d, slim.conv2d_transpose],
                            normalizer_fn=None,
                            weights_regularizer=slim.l2_regularizer(0.05),
                            activation_fn=tf.nn.relu):
            # cnv1 to cnv5b are shared between pose and explainability prediction
            cnv1  = slim.conv2d(inputs,16,  [7, 7], stride=2, scope='cnv1')
            cnv2  = slim.conv2d(cnv1, 32,  [5, 5], stride=2, scope='cnv2')
            cnv3  = slim.conv2d(cnv2, 64,  [3, 3], stride=2, scope='cnv3')
            cnv4  = slim.conv2d(cnv3, 128, [3, 3], stride=2, scope='cnv4')
            cnv5  = slim.conv2d(cnv4, 256, [3, 3], stride=2, scope='cnv5')
            # Pose specific layers
            with tf.variable_scope('pose'):
                cnv6  = slim.conv2d(cnv5, 256, [3, 3], stride=2, scope='cnv6')
                cnv7  = slim.conv2d(cnv6, 256, [3, 3], stride=2, scope='cnv7')
                pose_pred = slim.conv2d(cnv7, 12, [1, 1], scope='pred', 
                    stride=1, normalizer_fn=None, activation_fn=None)
                pose_avg = tf.reduce_mean(pose_pred, [1, 2])
                # Empirically we found that scaling by a small constant 
                # facilitates training.
                pose_final = tf.reshape(pose_avg, [-1, 12])
    return pose_final
    
    
def get_train_op_fn(loss, params):
    return tf.contrib.layers.optimize_loss(loss=loss, global_step=tf.contrib.framework.get_global_step(),
                                          optimizer=tf.train.AdamOptimizer, learning_rate=0.01)
    
def model_fn(features, labels, mode, params):
    is_training = mode == ModeKeys.TRAIN
    
    #predictions = nvidia_architecture(features, is_training=is_training)
    predictions = sfmlearner_architecture2(features, is_training=is_training)
        
    loss = None
    train_op = None
    eval_metric_ops = {}
    if mode != ModeKeys.INFER:
        loss = tf.reduce_mean(tf.losses.absolute_difference(labels, predictions), name='loss')
        #loss = tf.losses.mean_squared_error(labels=labels, predictions=predictions, scope='loss')
        train_op = get_train_op_fn(loss, params)
        
#     image_curr, image_prev = tf.split(features, 2, axis=3)
#     tf.summary.scalar('loss', loss)
#     tf.summary.image('prev_images', image_prev)
#     tf.summary.image('curr_images', image_curr)
#     tf.summary.image('diff_images', tf.abs(tf.subtract(image_prev, image_curr)))
        
    tensors_to_log = {'loss': loss}
    logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=500)
    train_hooks = [logging_hook]
    
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions, loss=loss, train_op=train_op, training_hooks=train_hooks)

In [4]:
HEIGHT = 128
WIDTH = 416
DEPTH = 1

class KittiDataset(object):
    def __init__(self, data_dir, subset='train', use_distortion=True):
        self.data_dir = data_dir
        self.subset = subset
        self.use_distortion = use_distortion
        
    def get_filenames(self):
        if self.subset in ['train', 'validation', 'eval']:
            return [os.path.join(self.data_dir, self.subset + '_sfm.tfrecord')]
            #return [os.path.join(self.data_dir, 'train/train00.odometry')]
        else:
            raise ValueError('Invalid data subset "%s"' % self.subset)
            
    def parser(self, record):
        keys_to_features = {
            "pose": tf.FixedLenFeature((), tf.string, default_value=""),
            "img_raw": tf.FixedLenFeature((), tf.string, default_value=""),
            "img_raw_prev": tf.FixedLenFeature((), tf.string, default_value=""),
        }
        parsed = tf.parse_single_example(record, keys_to_features)

        # Perform additional preprocessing on the parsed data.
        image = tf.image.decode_jpeg(parsed["img_raw"])
        image = tf.reshape(image, [HEIGHT, WIDTH, DEPTH])
        image = tf.cast(image, tf.float32)
        
        image_prev = tf.image.decode_jpeg(parsed["img_raw_prev"])
        image_prev = tf.reshape(image_prev, [HEIGHT, WIDTH, DEPTH])
        image_prev = tf.cast(image_prev, tf.float32)
        
        label = tf.decode_raw(parsed["pose"], tf.float64)
        label = tf.reshape(label, [12])
        
        return image, image_prev, label
    
    def make_batch(self, batch_size):
        filenames = self.get_filenames()
        
        dataset = tf.data.TFRecordDataset(filenames).repeat()
        
        # parse records
        dataset = dataset.map(self.parser)
        
        dataset = dataset.batch(batch_size)
        iterator = dataset.make_one_shot_iterator()
        image_batch, image_prev_batch, label_batch = iterator.get_next()
        
        image_batch = self.preprocess(image_batch, image_prev_batch)
        
        return image_batch, label_batch
    
    def preprocess(self, image, image_prev):
        image = tf.divide(tf.subtract(image, MEAN), STD)
        image_prev = tf.divide(tf.subtract(image_prev, MEAN), STD)
        image = tf.concat([image, image_prev], axis=3)
        #image = tf.image.resize_image_with_crop_or_pad(image, 25, 100)
        return image
    

    

In [5]:
def input_fn(data_dir, subset, batch_size, use_distortion_for_training=True):
    with tf.device('/cpu:0'):
        use_distortion = subset == 'train' and use_distortion_for_training
        dataset = KittiDataset(data_dir, subset, use_distortion)
        image_batch, label_batch = dataset.make_batch(batch_size)
        return image_batch, label_batch

In [6]:
import argparse
import functools
import itertools
import os

In [7]:
def get_experiment_fn(data_dir, use_distortion_for_training=True):
    def _experiment_fn(run_config, hparams):
        train_input_fn = functools.partial(input_fn, data_dir, subset='train', batch_size=16, use_distortion_for_training=True)
        eval_input_fn = functools.partial(input_fn, data_dir, subset='eval', batch_size=16)
        
        train_steps = 250000
        eval_steps = 200
        
        classifier = tf.estimator.Estimator(model_fn=model_fn, config=run_config, params=hparams)
        
        return tf.contrib.learn.Experiment(classifier, train_input_fn=train_input_fn,
                                          eval_input_fn=eval_input_fn, train_steps=train_steps,
                                          eval_steps=eval_steps)
    return _experiment_fn

In [ ]:
def run_experiment(data_dir, use_distortion_for_training):
    """Run the training experiment."""
    # Define model parameters
    params = tf.contrib.training.HParams(
        learning_rate=0.01,
        n_classes=12,
        train_steps=250000,
        min_eval_frequency=10000
    )

    # Set the run_config and the directory to save the model and stats
    run_config = tf.contrib.learn.RunConfig()
    run_config = run_config.replace(model_dir='./kitti_training_sfm')

    
    learn_runner.run(
        experiment_fn=get_experiment_fn(data_dir, use_distortion_for_training),  # First-class function
        run_config=run_config,  # RunConfig
        schedule="train_and_evaluate",  # What to run
        hparams=params  # HParams
    )

In [ ]:
tf.app.run(main=run_experiment( '/media/ryan/UBUNTU 16_0/', False))

INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f00773b0898>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': './kitti_training_sfm'}
Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.
Instructions for updating:
Please switch to tf.train.get_global_step
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ./kitti_training_sfm/model.ckpt.
INFO:tensorflow:Starting evaluation at 2018-01-08-19:52:20
INFO:tensorflow:

INFO:tensorflow:Evaluation [175/200]
INFO:tensorflow:Evaluation [176/200]
INFO:tensorflow:Evaluation [177/200]
INFO:tensorflow:Evaluation [178/200]
INFO:tensorflow:Evaluation [179/200]
INFO:tensorflow:Evaluation [180/200]
INFO:tensorflow:Evaluation [181/200]
INFO:tensorflow:Evaluation [182/200]
INFO:tensorflow:Evaluation [183/200]
INFO:tensorflow:Evaluation [184/200]
INFO:tensorflow:Evaluation [185/200]
INFO:tensorflow:Evaluation [186/200]
INFO:tensorflow:Evaluation [187/200]
INFO:tensorflow:Evaluation [188/200]
INFO:tensorflow:Evaluation [189/200]
INFO:tensorflow:Evaluation [190/200]
INFO:tensorflow:Evaluation [191/200]
INFO:tensorflow:Evaluation [192/200]
INFO:tensorflow:Evaluation [193/200]
INFO:tensorflow:Evaluation [194/200]
INFO:tensorflow:Evaluation [195/200]
INFO:tensorflow:Evaluation [196/200]
INFO:tensorflow:Evaluation [197/200]
INFO:tensorflow:Evaluation [198/200]
INFO:tensorflow:Evaluation [199/200]
INFO:tensorflow:Evaluation [200/200]
INFO:tensorflow:Finished evaluation at

INFO:tensorflow:Evaluation [114/200]
INFO:tensorflow:Evaluation [115/200]
INFO:tensorflow:Evaluation [116/200]
INFO:tensorflow:Evaluation [117/200]
INFO:tensorflow:Evaluation [118/200]
INFO:tensorflow:Evaluation [119/200]
INFO:tensorflow:Evaluation [120/200]
INFO:tensorflow:Evaluation [121/200]
INFO:tensorflow:Evaluation [122/200]
INFO:tensorflow:Evaluation [123/200]
INFO:tensorflow:Evaluation [124/200]
INFO:tensorflow:Evaluation [125/200]
INFO:tensorflow:Evaluation [126/200]
INFO:tensorflow:Evaluation [127/200]
INFO:tensorflow:Evaluation [128/200]
INFO:tensorflow:Evaluation [129/200]
INFO:tensorflow:Evaluation [130/200]
INFO:tensorflow:Evaluation [131/200]
INFO:tensorflow:Evaluation [132/200]
INFO:tensorflow:Evaluation [133/200]
INFO:tensorflow:Evaluation [134/200]
INFO:tensorflow:Evaluation [135/200]
INFO:tensorflow:Evaluation [136/200]
INFO:tensorflow:Evaluation [137/200]
INFO:tensorflow:Evaluation [138/200]
INFO:tensorflow:Evaluation [139/200]
INFO:tensorflow:Evaluation [140/200]
I